In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
        break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Starting slowly looking into data given  ***Note* I am a noobie**

In [ ]:
#improting essentials
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing import image_dataset_from_directory
import time
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from IPython.display import Image
from keras.applications.imagenet_utils import preprocess_input
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [ ]:
labels=pd.read_csv('/kaggle/input/happy-whale-and-dolphin/train.csv')
labels.head()
import os
if len(os.listdir("/kaggle/input/happy-whale-and-dolphin/train_images/"))==len(labels):
  print("proceed Further you are good to go")
else:
  print("some file may be missing check again!")

In [ ]:
#checking unique no of species in dataset
species=np.array(labels['species'])
print(np.unique(species),len(np.unique(species)))

In [ ]:
#cleaning some stuff in labels that we noticed in previous cells
labels['species']=labels['species'].apply(lambda x: 'bottlenose_dolphin' if x=='bottlenose_dolpin' else x)
labels['species']=labels['species'].apply(lambda x: 'killer_whale' if x=='kiler_whale' else x)
labels['species']=labels['species'].apply(lambda x: 'long_finned_pilot_whale' if x=='pilot_whale' else x)
species=np.array(labels['species'])
print(np.unique(species),len(np.unique(species)))

In [ ]:
labels['species'].value_counts().plot.bar(figsize=(10,3));

In [ ]:
#making a boolean matrix for a species
unique_species=np.unique(species)
species[0]==unique_species

In [ ]:
#like previous making boolean labels for all species  values
boolean_labels=[temp==unique_species for temp in species]
len(boolean_labels)

In [ ]:
#we can easily switch from boolean values to numbers
int_label=boolean_labels[0].astype(int)
int_label

**Starting looking into training images**

In [ ]:
#trying to get all images path in a variable
def give_image_paths(repo):
    image_paths=labels['image']
    image_paths=image_paths.apply(lambda x: '/kaggle/input/happy-whale-and-dolphin/'+str(repo)+'/'+str(x))
    return np.array(image_paths)
train_images=give_image_paths('train_images')
train_images[0],len(train_images)

In [ ]:
#Sneak peek at the training images
def read_one_image(path):
    testimg=tf.io.read_file(path)
    testimg=tf.image.decode_jpeg(testimg,channels=3)
    testimg=tf.image.convert_image_dtype(testimg,tf.float32)
    testimg=tf.image.resize(testimg,size=[256,256])
    return testimg
i=0
plt.figure(figsize=(15,15))
for i in range(25):
    testimg=read_one_image(train_images[i])
    ax=plt.subplot(5,5,i+1)
    i+=1
    plt.imshow(testimg)
    plt.title(species[i])
    plt.axis('off');

In [ ]:
#we will start experimenting on first few images
#NUM_IMAGES = 5000
NUM_IMAGES = 51033
X=train_images[:NUM_IMAGES]
y=boolean_labels[:NUM_IMAGES]
X_train,X_val,y_train,y_val=train_test_split(X[:NUM_IMAGES],y[:NUM_IMAGES] , test_size=0.1,random_state=42)
len(X_train),len(y_train),len(X_val),len(y_val)

In [ ]:
# for x in range(len(X)):
#     X[x]=read_one_image(X[x])

In [ ]:
# X[0]

**Lets turn our data into batches**

In [ ]:
# create a tuple of tensors
def get_image_label(image_path,label):
  """
  takes image path anmd label and give tensor image and label tuple
  """
  image=read_one_image(image_path)
  return image,label
batch_size=32
#create a function to turn data into batches
#we are gonna turn training , validation and test batch separately
def create_data_batches(X,y=None,batch_size=batch_size,valid_data=False,test_data=False):
  """
  create batches of data out of image (X) and label(y) pairs.
  Shuffles the data if its training but doesnt shuffle if its validation data
  """
  # if data is test data setwe know we dont have labels
  if test_data:
    print("Creating test data batches......")
    data=tf.data.Dataset.from_tensor_slices((tf.constant(X)))
    data_batch=data.map(read_one_image).batch(batch_size)
    return data_batch
  # if the data is validatuion dataset,we dont need to shuffle it
  elif valid_data:
    print("Creating validation data batches.....")
    data=tf.data.Dataset.from_tensor_slices((tf.constant(X),tf.constant(y)))
    data_batch=data.map(get_image_label).batch(batch_size)
    return data_batch
  # if its taring data
  else:
    print("Creating training data batches......")
    #turn file path and labels into tensors
    data=tf.data.Dataset.from_tensor_slices((tf.constant(X),tf.constant(y)))
    #shuffling data
    data=data.shuffle(buffer_size=len(X))
    data=data.map(get_image_label)
    #turn trainig data into batches
    data_batch=data.batch(batch_size)
    return data_batch

In [ ]:
#create training and validataion data batches
train_data=create_data_batches(X_train,y_train)
val_data=create_data_batches(X_val,y_val,valid_data=True)
train_data.element_spec,val_data.element_spec

# Model building starts

In [ ]:
# lets get our input shape and output shape cleared to build a model
NUM_SIZE=256
input_shape=[None,NUM_SIZE,NUM_SIZE,3]
output_shape=len(unique_species)
input_shape,output_shape

In [ ]:
#creating Neural Network
#First add CNN part as base
model=Sequential()
model.add(layers.Conv2D(5,3,3,activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Flatten())
#Adding ANN for full connection
model.add(layers.Dense(output_shape,activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])


In [ ]:
#lets train you can play around with batchsize and epochs
history=model.fit(train_data,validation_data=val_data,batch_size=32,epochs=3)

In [ ]:
model.save('/kaggle/working/version1.h5')

In [ ]:
plt.plot(history.history['loss'])

In [ ]:
plt.plot(history.history['accuracy'])

In [ ]:
pred=model.predict(np.expand_dims(read_one_image(train_images[26]),axis=0))

In [ ]:
pred

In [ ]:
np.argmax(pred),np.argmax()

In [ ]:
unique_species[23]

In [ ]:
print(os.listdir('/kaggle/working/'))

In [ ]:
from keras import models
model_loaded=models.load_model('/kaggle/working/version1.h5')

In [ ]:
#read test data
test_images=[]
for dirname, _, filenames in os.walk('/kaggle/input/happy-whale-and-dolphin/test_images/'):
    test_images=filenames
test_images[:10]

In [ ]:
sample=pd.read_csv('/kaggle/input/happy-whale-and-dolphin/sample_submission.csv')
sample.head()

In [ ]:
output=pd.DataFrame()
image=[]
predictions=[]
for i in range(len(test_images)):
    test_pred=model_loaded.predict(np.expand_dims(read_one_image("/kaggle/input/happy-whale-and-dolphin/test_images/"+str(test_images[i])),axis=0))
    val=test_pred.argmax()
    species_predicted=unique_species[val]
    temp=labels[labels['species']==species_predicted][:5]
    image.append(test_images[i])
    predictions.append(' '.join(temp['individual_id']))
output['image']=image
output['predictions']=predictions